<td>   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a></td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/project_configuration/multimodal_chat_project.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/project_configuration/multimodal_chat_project.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Multimodal chat project setup

This notebook will provide an example workflow of setting up a multimodal Chat (MMC) Project with the Labelbox-Python SDK.
Multimodal Chat Projects are set up differently than other projects with its own unique method and modifications to existing methods:

- `client.create_model_evaluation_project`: The main method used to create a live multimodal Chat project.
  
- `client.create_offline_model_evaluation_project`: The main method used to create a offline multimodal Chat project.

- `client.create_ontology`: Methods used to create Labelbox ontologies for LMC project this requires an `ontology_kind` parameter set to `lb.OntologyKind.ModelEvaluation`.

- `client.create_ontology_from_feature_schemas`: Similar to `client.create_ontology` but from a list of `feature schema ids` designed to allow you to use existing features instead of creating new features. This also requires an `ontology_kind` set to `lb.OntologyKind.ModelEvaluation`.

## Set up

In [ ]:
%pip install -q --upgrade "labelbox[data]"

In [ ]:
import labelbox as lb

## API key and client
Please provide a valid API key below to connect to the Labelbox client properly. For more information, please review the [Create API key guide](https://docs.labelbox.com/reference/create-api-key).

In [ ]:
API_KEY = None
client = lb.Client(api_key=API_KEY)

## Example: Create multimodal Chat project

The steps to creating a multimodal Chat Projects through the Labelbox-Python SDK are similar to creating a regular project. However, they vary slightly, and we will showcase the different methods in this example workflow.

### Create a multimodal chat ontology

You can create ontologies for multimodal chat projects in the same way as other project ontologies using two methods: `client.create_ontology` and `client.create_ontology_from_feature_schemas`. The only additional requirement is to pass an ontology_kind parameter, which needs to be set to `lb.OntologyKind.ModelEvaluation`.

#### Option A: `client.create_ontology`

Typically, you create ontologies and generate the associated features simultaneously. Below is an example of creating an ontology for your multimodal chat project using supported tools and classifications; for information on supported annotation types, visit our [multimodal chat evaluation guide](https://docs.labelbox.com/docs/multimodal-chat#supported-annotation-types) guide.

In [ ]:
ontology_builder = lb.OntologyBuilder(
    tools=[
        lb.Tool(
            tool=lb.Tool.Type.MESSAGE_SINGLE_SELECTION,
            name="single select feature",
        ),
        lb.Tool(
            tool=lb.Tool.Type.MESSAGE_MULTI_SELECTION,
            name="multi select feature",
        ),
        lb.Tool(tool=lb.Tool.Type.MESSAGE_RANKING, name="ranking feature"),
    ],
    classifications=[
        lb.Classification(
            class_type=lb.Classification.Type.CHECKLIST,
            name="checklist feature",
            options=[
                lb.Option(value="option 1", label="option 1"),
                lb.Option(value="option 2", label="option 2"),
            ],
        ),
        lb.Classification(
            class_type=lb.Classification.Type.RADIO,
            name="radio_question",
            options=[
                lb.Option(value="first_radio_answer"),
                lb.Option(value="second_radio_answer"),
            ],
        ),
    ],
)

# Create ontology
ontology = client.create_ontology(
    "LMC ontology",
    ontology_builder.asdict(),
    media_type=lb.MediaType.Conversational,
    ontology_kind=lb.OntologyKind.ModelEvaluation,
)

### Option B: `client.create_ontology_from_feature_schemas`
Ontologies can also be created with feature schema IDs. This makes your ontologies with existing features compared to generating new features. You can get these features by going to the _Schema_ tab inside Labelbox. (uncomment the below code block for this option)

In [ ]:
# ontology = client.create_ontology_from_feature_schemas(
#     "LMC ontology",
#     feature_schema_ids=["<list of feature schema ids"],
#     media_type=lb.MediaType.Conversational,
#     ontology_kind=lb.OntologyKind.ModelEvaluation,
# )

## Creating multimodal chat evaluation projects

There are two versions of a multimodal chat evaluation projects:

1. Offline multimodal chat evaluation projects: Data rows will need to be imported manually and have no live model invocation.

2. Live multimodal chat evaluation projects: Empty data rows are generated on project creation and are filled out with live model invocation.

We will discuss creating both types of projects with the Labelbox SDK.

## Set up offline multimodal chat evaluation project

For an offline multimodal chat evaluation project, you must import conversational version 2 data rows. For more information, please visit our [import multimodal chat evaluation data](https://docs.labelbox.com/reference/import-multimodal-chat-data) guide. Offline multimodal chat evaluation projects are created through the SDK with `client.create_offline_model_evaluation_project`. This method uses the same parameters as `client.create_project` but provides better validation to ensure the project is set up correctly.

In [ ]:
project = client.create_offline_model_evaluation_project(
    name="<project_name>",
    description="<project_description>",  # optional
)

### Set Up Live Multimodal Chat project
You do not have to create data rows with a model evaluation project; instead, they are generated for you when you create the project. The method you use to create your project is `client.create_model_evaluation_project`, which takes the same parameters as the traditional `client.create_project` but with a few specific additional parameters. 

#### Parameters
When using `client.create_model_evaluation_project` the following parameters are needed:

- `create_model_evaluation_project` parameters:

    - `name`: The name of your new project.

    - `description`: An optional description of your project.

    - `media_type`: The type of assets that this project will accept. This should be set to lb.MediaType.Conversational

    - `dataset_name`: The name of the dataset where the generated data rows will be located. Include this parameter only if you want to create a new dataset.

    - `dataset_id`: An optional dataset ID of an existing Labelbox dataset. Include this parameter if you are wanting to append to an existing LMC dataset.

    - `data_row_count`: The number of data row assets that will be generated and used with your project.


In [ ]:
project = client.create_model_evaluation_project(
    name="Demo LMC Project",
    media_type=lb.MediaType.Conversational,
    dataset_name="Demo LMC dataset",
    data_row_count=100,
)

# Setup project with ontology created above
project.setup_editor(ontology)

## Setting up model config
You can create, delete, attach and remove model configs from your Live Multimodal Chat project through the Labelbox-Python SDK. These are the model configs that you will be evaluating for your responses. 

### Creating model config
The main method associated with creating a model config is `client.create_model_config`. This method takes the following parameters:

- `name`: Name of the model config.

- `model_id`: The ID of the model to configure. You must obtain this through the UI by navigating to the Model tab, selecting the model you are trying to use, and copying the id inside the URL. For supported models, visit the [Live Multimodal Chat page](https://docs.labelbox.com/docs/live-multimodal-chat#supported-annotation-types).

- `inference_params`: JSON of model configuration parameters. This will vary depending on the model you are trying to set up. It is recommended to first set up a model config inside the UI to learn all the associated parameters.

For the example below, we will be setting up a Google Gemini 1.5 Pro model config.

In [ ]:
MODEL_ID = "270a24ba-b983-40d6-9a1f-98a1bbc2fb65"

inference_params = {"max_new_tokens": 1024, "use_attachments": True}

model_config = client.create_model_config(
    name="Example model config",
    model_id=MODEL_ID,
    inference_params=inference_params,
)

### Attaching model config to project
You can attach and remove model configs to your project using `project.add_model_config` or `project.remove_model_config`. Both methods take just a `model_config` ID.

In [ ]:
project.add_model_config(model_config.uid)

### Delete model config
You can also delete model configs using the `client.delete_model_config`. You just need to pass in the `model_config` ID in order to delete your model config. You can obtain this ID from your created model config above or get the model configs directly from your project using `project.project_model_configs` and then iterating through the list of model configs attached to your project. Uncomment the code below to delete your model configs. 

In [ ]:
# model_configs = project.project_model_configs()

# for model_config in model_configs:
#     client.delete_model_config(model_config.uid)

### Mark project setup as completed

Once you have finalized your project and set up your model configs, you must mark the project setup as completed.

**Once the project is marked as "setup complete", a user can not add, modify, or delete existing project model configs.**

In [ ]:
project.set_project_model_setup_complete()

## Exporting Live Multimodal Chat project
Exporting from a Live Multimodal Chat project works the same as exporting from other projects. In this example, your export will be shown as empty unless you have created labels inside the Labelbox platform. Please review our [Live Multimodal Chat Export](https://docs.labelbox.com/reference/export-live-multimodal-chat-annotations) guide for a sample export.

In [ ]:
# Start export from project
export_task = project.export()
export_task.wait_till_done()

# Conditional if task has errors
if export_task.has_errors():
    export_task.get_buffered_stream(stream_type=lb.StreamType.ERRORS).start(
        stream_handler=lambda error: print(error))

if export_task.has_result():
    # Start export stream
    stream = export_task.get_buffered_stream()

    # Iterate through data rows
    for data_row in stream:
        print(data_row.json)

## Clean up

This section serves as an optional clean-up step to delete the Labelbox assets created within this guide. You will need to uncomment the delete methods shown.

In [ ]:
# project.delete()
# client.delete_unused_ontology(ontology.uid)
# dataset.delete()